<a href="https://colab.research.google.com/github/maclandrol/cours-ia-med/blob/master/07_nnUNet_Clinical_Workflows.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 07. nnU-Net - Guide Complet et Workflows Cliniques

**Enseignant:** Emmanuel Noutahi, PhD

---

**Objectif:** Maîtriser nnU-Net de l'installation au déploiement clinique.

**Applications pratiques :**
- Configuration complète de l'environnement nnU-Net
- Entraînement sur datasets médicaux (Medical Decathlon)
- Adaptation aux données personnalisées
- Inférence et post-traitement
- Intégration dans workflows hospitaliers (DICOM, PACS)
- Déploiement en production clinique

**Important:** Ce cours couvre l'implémentation complète de nnU-Net pour usage clinique réel.

## Installation et Configuration

nnU-Net est un framework de segmentation automatique qui s'adapte automatiquement aux données d'entraînement.

In [ ]:
# Installation complète de nnU-Net et dépendances
!pip install nnunet torch torchvision -q
!pip install nibabel SimpleITK pandas numpy matplotlib seaborn -q
!pip install scikit-learn scipy tqdm -q
!pip install pydicom monai -q

import os
import sys
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import json
import warnings
warnings.filterwarnings('ignore')

# Configuration système
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Dispositif utilisé: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Mémoire GPU: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

print(f"PyTorch version: {torch.__version__}")
print("Configuration nnU-Net prête.")

## 1. Configuration de l'Environnement nnU-Net

nnU-Net utilise une structure de dossiers spécifique pour organiser les données d'entraînement et les résultats.

In [ ]:
# Configuration de l'environnement nnU-Net
print("=== CONFIGURATION DE L'ENVIRONNEMENT NNUNET ===")

# Configuration des chemins nnU-Net
base_dir = Path("/content/nnUNet_data") if 'google.colab' in sys.modules else Path("./nnUNet_data")

# Structure des dossiers nnU-Net
folders = {
    'nnUNet_raw': base_dir / "nnUNet_raw",
    'nnUNet_preprocessed': base_dir / "nnUNet_preprocessed", 
    'nnUNet_results': base_dir / "nnUNet_results",
    'nnUNet_trained_models': base_dir / "nnUNet_trained_models"
}

# Création des dossiers
for folder_name, folder_path in folders.items():
    folder_path.mkdir(parents=True, exist_ok=True)
    print(f"Dossier créé: {folder_path}")

# Configuration des variables d'environnement
os.environ['nnUNet_raw'] = str(folders['nnUNet_raw'])
os.environ['nnUNet_preprocessed'] = str(folders['nnUNet_preprocessed'])
os.environ['nnUNet_results'] = str(folders['nnUNet_results'])

print(f"\nVariables d'environnement configurées:")
print(f"nnUNet_raw: {os.environ.get('nnUNet_raw')}")
print(f"nnUNet_preprocessed: {os.environ.get('nnUNet_preprocessed')}")
print(f"nnUNet_results: {os.environ.get('nnUNet_results')}")

# Fonction utilitaire pour créer la structure dataset
def create_dataset_structure(dataset_id, dataset_name, base_folder):
    """
    Crée la structure de dossiers pour un dataset nnU-Net
    """
    dataset_folder = base_folder / f"Dataset{dataset_id:03d}_{dataset_name}"
    
    subfolders = ['imagesTr', 'labelsTr', 'imagesTs', 'labelsTs']
    created_paths = {}
    
    for subfolder in subfolders:
        path = dataset_folder / subfolder
        path.mkdir(parents=True, exist_ok=True)
        created_paths[subfolder] = path
    
    # Fichier dataset.json template
    dataset_json = {
        "name": dataset_name,
        "description": f"Dataset {dataset_id} - {dataset_name}",
        "tensorImageSize": "3D",
        "reference": "Custom dataset",
        "licence": "Educational use",
        "release": "1.0",
        "modality": {
            "0": "MRI"
        },
        "labels": {
            "0": "Background",
            "1": "Target"
        },
        "numTraining": 0,
        "numTest": 0,
        "training": [],
        "test": []
    }
    
    json_path = dataset_folder / "dataset.json"
    with open(json_path, 'w') as f:
        json.dump(dataset_json, f, indent=2)
    
    created_paths['dataset_json'] = json_path
    created_paths['dataset_folder'] = dataset_folder
    
    return created_paths

print("\nFonctions utilitaires créées.")
print("Environnement nnU-Net configuré avec succès.")

## 2. Préparation des Données Médicales

Préparons des données d'exemple au format nnU-Net en utilisant le Medical Decathlon comme référence.

In [ ]:
# Préparation des données médicales pour nnU-Net
print("=== PRÉPARATION DES DONNÉES MÉDICALES ===")

import nibabel as nib
import SimpleITK as sitk

def create_synthetic_medical_data(task_name="Hippocampus", task_id=4, num_samples=5):
    """
    Crée des données médicales synthétiques au format nnU-Net
    Simule le dataset Medical Decathlon Task004 - Hippocampus
    """
    
    # Création de la structure
    dataset_paths = create_dataset_structure(task_id, task_name, folders['nnUNet_raw'])
    
    print(f"Création de {num_samples} échantillons synthétiques...")
    
    training_cases = []
    
    for i in range(num_samples):
        case_id = f"hippocampus_{i+1:03d}"
        
        # Création d'une image cérébrale synthétique (64x64x32)
        image_shape = (64, 64, 32)
        
        # Image de base (matière grise/blanche)
        brain_image = np.random.normal(100, 20, image_shape).astype(np.float32)
        
        # Ajout du crâne
        x, y, z = np.meshgrid(
            np.linspace(-1, 1, image_shape[0]),
            np.linspace(-1, 1, image_shape[1]), 
            np.linspace(-1, 1, image_shape[2]),
            indexing='ij'
        )
        
        # Masque cérébral (sphère)
        brain_mask = (x**2 + y**2 + z**2) < 0.8
        brain_image[~brain_mask] = 0
        
        # Création de l'hippocampe synthétique
        hippo_mask = np.zeros(image_shape, dtype=np.uint8)
        
        # Hippocampe gauche
        hippo_left = (
            (x + 0.3)**2 * 4 + (y - 0.1)**2 * 8 + (z)**2 * 12 < 0.15
        ) & brain_mask
        
        # Hippocampe droit 
        hippo_right = (
            (x - 0.3)**2 * 4 + (y - 0.1)**2 * 8 + (z)**2 * 12 < 0.15
        ) & brain_mask
        
        # Assignation des labels
        hippo_mask[hippo_left] = 1  # Hippocampe gauche
        hippo_mask[hippo_right] = 2  # Hippocampe droit
        
        # Intensité plus élevée dans l'hippocampe
        brain_image[hippo_mask > 0] = 150 + np.random.normal(0, 10, np.sum(hippo_mask > 0))
        
        # Sauvegarde au format NIfTI
        # Image principale
        img_filename = f"{case_id}_0000.nii.gz"
        img_path = dataset_paths['imagesTr'] / img_filename
        
        # Création du NIfTI avec métadonnées réalistes
        affine = np.eye(4)
        affine[0, 0] = affine[1, 1] = affine[2, 2] = 1.0  # Résolution 1mm isotrope
        
        nifti_img = nib.Nifti1Image(brain_image, affine)
        nib.save(nifti_img, img_path)
        
        # Label/segmentation
        label_filename = f"{case_id}.nii.gz"
        label_path = dataset_paths['labelsTr'] / label_filename
        
        nifti_label = nib.Nifti1Image(hippo_mask.astype(np.uint8), affine)
        nib.save(nifti_label, label_path)
        
        # Ajout à la liste d'entraînement
        training_cases.append({
            "image": f"./imagesTr/{img_filename}",
            "label": f"./labelsTr/{label_filename}"
        })
        
        print(f"  Cas {i+1}/{num_samples} créé: {case_id}")
    
    # Mise à jour du dataset.json
    with open(dataset_paths['dataset_json'], 'r') as f:
        dataset_json = json.load(f)
    
    dataset_json.update({
        "description": "Synthetic Hippocampus segmentation from brain MRI",
        "modality": {
            "0": "T1"
        },
        "labels": {
            "0": "Background",
            "1": "Hippocampus_left", 
            "2": "Hippocampus_right"
        },
        "numTraining": len(training_cases),
        "training": training_cases
    })
    
    with open(dataset_paths['dataset_json'], 'w') as f:
        json.dump(dataset_json, f, indent=2)
    
    return dataset_paths, training_cases

# Création du dataset d'exemple
dataset_paths, training_data = create_synthetic_medical_data(
    task_name="Hippocampus", 
    task_id=4, 
    num_samples=8
)

print(f"\nDataset créé avec succès:")
print(f"Dossier: {dataset_paths['dataset_folder']}")
print(f"Images d'entraînement: {len(training_data)}")

# Vérification de la structure
def verify_dataset_structure(dataset_folder):
    """
    Vérifie la structure du dataset nnU-Net
    """
    print(f"\nVérification de la structure du dataset:")
    
    required_folders = ['imagesTr', 'labelsTr']
    for folder in required_folders:
        folder_path = dataset_folder / folder
        if folder_path.exists():
            file_count = len(list(folder_path.glob('*.nii.gz')))
            print(f"✓ {folder}: {file_count} fichiers")
        else:
            print(f"✗ {folder}: Dossier manquant")
    
    # Vérification du dataset.json
    json_path = dataset_folder / "dataset.json"
    if json_path.exists():
        with open(json_path, 'r') as f:
            data = json.load(f)
        print(f"✓ dataset.json: {data.get('numTraining', 0)} cas d'entraînement")
        print(f"  Modalités: {list(data.get('modality', {}).values())}")
        print(f"  Labels: {list(data.get('labels', {}).values())}")
    else:
        print(f"✗ dataset.json: Fichier manquant")

verify_dataset_structure(dataset_paths['dataset_folder'])

print("\nDonnées préparées pour nnU-Net.")

## 3. Préprocessing et Configuration Automatique

nnU-Net analyse automatiquement les données et configure les paramètres optimaux d'entraînement.

In [ ]:
# Préprocessing automatique nnU-Net
print("=== PRÉPROCESSING ET CONFIGURATION AUTOMATIQUE ===")

def run_nnunet_preprocessing(dataset_id, dataset_folder):
    """
    Simule le préprocessing nnU-Net
    En production, ceci utiliserait: nnUNetv2_plan_and_preprocess
    """
    
    print(f"Analyse du dataset {dataset_id}...")
    
    # Simulation de l'analyse des données
    imagesTr_folder = dataset_folder / "imagesTr"
    labelsTr_folder = dataset_folder / "labelsTr"
    
    image_files = list(imagesTr_folder.glob('*.nii.gz'))
    label_files = list(labelsTr_folder.glob('*.nii.gz'))
    
    print(f"Images trouvées: {len(image_files)}")
    print(f"Labels trouvés: {len(label_files)}")
    
    # Analyse des propriétés des images
    image_properties = []
    
    for img_file in image_files[:3]:  # Analyser quelques échantillons
        try:
            nifti_img = nib.load(img_file)
            data = nifti_img.get_fdata()
            
            properties = {
                'shape': data.shape,
                'spacing': nifti_img.header.get_zooms(),
                'intensity_mean': float(np.mean(data[data > 0])),
                'intensity_std': float(np.std(data[data > 0])),
                'intensity_min': float(np.min(data)),
                'intensity_max': float(np.max(data))
            }
            
            image_properties.append(properties)
            
        except Exception as e:
            print(f"Erreur lors de l'analyse de {img_file}: {e}")
    
    # Calcul des statistiques globales
    if image_properties:
        avg_shape = tuple(int(np.mean([p['shape'][i] for p in image_properties])) 
                         for i in range(len(image_properties[0]['shape'])))
        avg_spacing = tuple(np.mean([p['spacing'][i] for p in image_properties]) 
                           for i in range(len(image_properties[0]['spacing'])))
        avg_intensity = np.mean([p['intensity_mean'] for p in image_properties])
        
        print(f"\nPropriétés moyennes du dataset:")
        print(f"  Forme moyenne: {avg_shape}")
        print(f"  Espacement moyen: {[f'{s:.2f}mm' for s in avg_spacing]}")
        print(f"  Intensité moyenne: {avg_intensity:.1f}")
    
    # Configuration automatique simulée
    configurations = {
        '2d': {
            'description': 'Configuration 2D - slice par slice',
            'patch_size': [256, 256],
            'batch_size': 12,
            'recommended_for': 'Images avec peu de contexte 3D'
        },
        '3d_fullres': {
            'description': 'Configuration 3D pleine résolution',
            'patch_size': [128, 128, 64] if avg_shape[2] < 100 else [96, 96, 96],
            'batch_size': 2,
            'recommended_for': 'Images 3D de taille modérée'
        },
        '3d_lowres': {
            'description': 'Configuration 3D basse résolution',
            'patch_size': [64, 64, 64],
            'batch_size': 4,
            'recommended_for': 'Images 3D de grande taille'
        },
        '3d_cascade': {
            'description': 'Configuration cascade (lowres → fullres)',
            'patch_size': 'Variable selon étape',
            'batch_size': 'Variable selon étape', 
            'recommended_for': 'Images très grandes, meilleure précision'
        }
    }
    
    print(f"\nConfigurations automatiques générées:")
    for config_name, config in configurations.items():
        print(f"\n  {config_name.upper()}:")
        print(f"    Description: {config['description']}")
        print(f"    Taille de patch: {config['patch_size']}")
        print(f"    Taille de batch: {config['batch_size']}")
        print(f"    Recommandé pour: {config['recommended_for']}")
    
    # Création du dossier preprocessed
    preprocessed_folder = folders['nnUNet_preprocessed'] / f"Dataset{dataset_id:03d}_Hippocampus"
    preprocessed_folder.mkdir(parents=True, exist_ok=True)
    
    # Sauvegarde des configurations
    config_file = preprocessed_folder / "nnUNetPlans.json"
    
    plans = {
        "dataset_name": f"Dataset{dataset_id:03d}_Hippocampus",
        "plans_name": "nnUNetPlans",
        "original_median_spacing_after_transp": list(avg_spacing),
        "original_median_shape_after_transp": list(avg_shape),
        "image_reader_writer": "SimpleITKIO",
        "transpose_forward": [0, 1, 2],
        "transpose_backward": [0, 1, 2],
        "configurations": configurations
    }
    
    with open(config_file, 'w') as f:
        json.dump(plans, f, indent=2)
    
    print(f"\nFichier de configuration sauvegardé: {config_file}")
    
    return plans, preprocessed_folder

# Exécution du preprocessing
dataset_id = 4
plans, preprocessed_folder = run_nnunet_preprocessing(dataset_id, dataset_paths['dataset_folder'])

print(f"\nPréprocessing terminé.")
print(f"Configuration recommandée pour ce dataset: 3d_fullres")
print(f"Raison: Images de taille modérée ({plans['original_median_shape_after_transp']})")

# Simulation du préprocessing des données
def simulate_data_preprocessing(source_folder, target_folder, configuration='3d_fullres'):
    """
    Simule le préprocessing des données (normalization, resampling, etc.)
    """
    print(f"\nSimulation du préprocessing des données ({configuration})...")
    
    # Création des dossiers de sortie
    config_folder = target_folder / configuration
    config_folder.mkdir(parents=True, exist_ok=True)
    
    # Simulation des fichiers préprocessés
    image_files = list((source_folder / "imagesTr").glob('*.nii.gz'))
    
    preprocessed_info = {
        'configuration': configuration,
        'num_files_processed': len(image_files),
        'normalization': 'Z-score normalization',
        'resampling': 'Isotropic 1mm spacing',
        'cropping': 'Non-zero region extraction',
        'output_format': 'NPZ compressed arrays'
    }
    
    # Sauvegarde des informations de preprocessing
    info_file = config_folder / "preprocessing_info.json"
    with open(info_file, 'w') as f:
        json.dump(preprocessed_info, f, indent=2)
    
    print(f"  Fichiers traités: {preprocessed_info['num_files_processed']}")
    print(f"  Normalisation: {preprocessed_info['normalization']}")
    print(f"  Rééchantillonnage: {preprocessed_info['resampling']}")
    print(f"  Format de sortie: {preprocessed_info['output_format']}")
    
    return preprocessed_info

# Simulation du preprocessing
preprocessing_info = simulate_data_preprocessing(
    dataset_paths['dataset_folder'], 
    preprocessed_folder,
    '3d_fullres'
)

print("\nPhase de préprocessing complétée.")

## 4. Entraînement du Modèle nnU-Net

Simulons l'entraînement nnU-Net avec monitoring des performances.

In [ ]:
# Entraînement du modèle nnU-Net
print("=== ENTRAÎNEMENT DU MODÈLE NNUNET ===")

import time
from datetime import datetime, timedelta

class nnUNetTrainingSimulator:
    """
    Simule l'entraînement nnU-Net avec monitoring réaliste
    """
    
    def __init__(self, dataset_id, configuration='3d_fullres', fold=0):
        self.dataset_id = dataset_id
        self.configuration = configuration
        self.fold = fold
        self.trainer_name = "nnUNetTrainer"
        
        # Paramètres d'entraînement simulés
        self.max_epochs = 1000
        self.initial_lr = 0.01
        self.current_epoch = 0
        
        # Métriques d'entraînement
        self.train_losses = []
        self.val_losses = []
        self.val_dice_scores = []
        self.learning_rates = []
        
        # Simulation de l'architecture
        self.network_architecture = {
            'name': 'nnU-Net',
            'encoder': 'ResNet encoder with instance normalization',
            'decoder': 'U-Net decoder with skip connections',
            'deep_supervision': True,
            'loss_function': 'Dice + Cross-entropy',
            'optimizer': 'SGD with momentum 0.99',
            'data_augmentation': 'Rotation, scaling, gamma, noise'
        }
    
    def simulate_training_epoch(self, epoch):
        """
        Simule une epoch d'entraînement
        """
        # Simulation réaliste des courbes d'apprentissage
        
        # Loss décroissante avec fluctuations
        base_train_loss = 0.8 * np.exp(-epoch / 100) + 0.1
        train_loss = base_train_loss + np.random.normal(0, 0.05)
        
        base_val_loss = 0.85 * np.exp(-epoch / 120) + 0.15
        val_loss = base_val_loss + np.random.normal(0, 0.08)
        
        # Dice score croissant
        base_dice = 0.95 * (1 - np.exp(-epoch / 80))
        val_dice = base_dice + np.random.normal(0, 0.05)
        val_dice = np.clip(val_dice, 0, 1)
        
        # Learning rate avec decay
        if epoch < 100:
            lr = self.initial_lr
        elif epoch < 250:
            lr = self.initial_lr * 0.1
        else:
            lr = self.initial_lr * 0.01
        
        return {
            'epoch': epoch,
            'train_loss': max(0.05, train_loss),
            'val_loss': max(0.1, val_loss),
            'val_dice': val_dice,
            'learning_rate': lr,
            'time_per_epoch': 2.5 + np.random.normal(0, 0.3)  # minutes
        }
    
    def run_training_simulation(self, num_epochs=300, print_frequency=50):
        """
        Lance la simulation d'entraînement
        """
        print(f"Démarrage entraînement nnU-Net:")
        print(f"Dataset: {self.dataset_id}, Configuration: {self.configuration}, Fold: {self.fold}")
        print(f"Architecture: {self.network_architecture['name']}")
        print(f"Optimiseur: {self.network_architecture['optimizer']}")
        print(f"Fonction de perte: {self.network_architecture['loss_function']}")
        print(f"Epochs prévues: {num_epochs}")
        
        start_time = time.time()
        best_val_dice = 0
        best_epoch = 0
        patience_counter = 0
        
        print(f"\n{'Epoch':<6} {'Train Loss':<12} {'Val Loss':<10} {'Val Dice':<10} {'LR':<10} {'Time/Epoch':<12}")
        print("-" * 70)
        
        for epoch in range(1, num_epochs + 1):
            # Simulation d'une epoch
            epoch_results = self.simulate_training_epoch(epoch)
            
            # Stockage des métriques
            self.train_losses.append(epoch_results['train_loss'])
            self.val_losses.append(epoch_results['val_loss'])
            self.val_dice_scores.append(epoch_results['val_dice'])
            self.learning_rates.append(epoch_results['learning_rate'])
            
            # Vérification du meilleur modèle
            if epoch_results['val_dice'] > best_val_dice:
                best_val_dice = epoch_results['val_dice']
                best_epoch = epoch
                patience_counter = 0
            else:
                patience_counter += 1
            
            # Affichage périodique
            if epoch % print_frequency == 0 or epoch == 1:
                print(f"{epoch:<6} {epoch_results['train_loss']:<12.4f} {epoch_results['val_loss']:<10.4f} "
                      f"{epoch_results['val_dice']:<10.4f} {epoch_results['learning_rate']:<10.6f} "
                      f"{epoch_results['time_per_epoch']:<12.1f} min")
            
            # Early stopping simulation
            if patience_counter > 100 and epoch > 200:  # Patience de 100 epochs
                print(f"\nEarly stopping à l'epoch {epoch} (patience dépassée)")
                break
        
        total_time = time.time() - start_time
        
        # Résumé de l'entraînement
        print(f"\n{'='*70}")
        print(f"ENTRAÎNEMENT TERMINÉ")
        print(f"{'='*70}")
        print(f"Epochs complétées: {len(self.train_losses)}")
        print(f"Meilleur Dice score: {best_val_dice:.4f} (epoch {best_epoch})")
        print(f"Temps total (simulé): {total_time/60:.1f} minutes")
        print(f"Loss finale train: {self.train_losses[-1]:.4f}")
        print(f"Loss finale validation: {self.val_losses[-1]:.4f}")
        
        # Sauvegarde des résultats
        results_folder = folders['nnUNet_results'] / f"Dataset{self.dataset_id:03d}_Hippocampus" / \
                        f"{self.trainer_name}__{self.configuration}__fold_{self.fold}"
        results_folder.mkdir(parents=True, exist_ok=True)
        
        training_log = {
            'dataset_id': self.dataset_id,
            'configuration': self.configuration,
            'fold': self.fold,
            'trainer': self.trainer_name,
            'architecture': self.network_architecture,
            'total_epochs': len(self.train_losses),
            'best_epoch': best_epoch,
            'best_val_dice': float(best_val_dice),
            'final_train_loss': float(self.train_losses[-1]),
            'final_val_loss': float(self.val_losses[-1]),
            'train_losses': [float(x) for x in self.train_losses],
            'val_losses': [float(x) for x in self.val_losses],
            'val_dice_scores': [float(x) for x in self.val_dice_scores],
            'learning_rates': [float(x) for x in self.learning_rates]
        }
        
        log_file = results_folder / "training_log.json"
        with open(log_file, 'w') as f:
            json.dump(training_log, f, indent=2)
        
        print(f"Log d'entraînement sauvegardé: {log_file}")
        
        return training_log, results_folder

# Lancement de l'entraînement simulé
trainer = nnUNetTrainingSimulator(dataset_id=4, configuration='3d_fullres', fold=0)
training_log, results_folder = trainer.run_training_simulation(num_epochs=250, print_frequency=50)

print(f"\nEntraînement nnU-Net simulé terminé.")
print(f"Modèle entraîné disponible dans: {results_folder}")

## 5. Visualisation des Performances d'Entraînement

Analysons les courbes d'apprentissage et les métriques de performance.

In [ ]:
# Visualisation des performances d'entraînement
print("=== VISUALISATION DES PERFORMANCES ===")

def plot_training_metrics(training_log):
    """
    Crée des graphiques de performance d'entraînement
    """
    epochs = list(range(1, len(training_log['train_losses']) + 1))
    
    # Configuration de la figure
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle(f"Performances d'Entraînement nnU-Net - Dataset {training_log['dataset_id']}", 
                fontsize=16, fontweight='bold')
    
    # 1. Courbes de loss
    axes[0, 0].plot(epochs, training_log['train_losses'], 'b-', label='Training Loss', alpha=0.8)
    axes[0, 0].plot(epochs, training_log['val_losses'], 'r-', label='Validation Loss', alpha=0.8)
    axes[0, 0].set_xlabel('Epoch')
    axes[0, 0].set_ylabel('Loss')
    axes[0, 0].set_title('Évolution des Loss')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)
    axes[0, 0].set_yscale('log')
    
    # 2. Score Dice de validation
    axes[0, 1].plot(epochs, training_log['val_dice_scores'], 'g-', label='Validation Dice', linewidth=2)
    axes[0, 1].axhline(y=training_log['best_val_dice'], color='r', linestyle='--', 
                      label=f'Meilleur: {training_log["best_val_dice"]:.4f}')
    axes[0, 1].set_xlabel('Epoch')
    axes[0, 1].set_ylabel('Dice Score')
    axes[0, 1].set_title('Score Dice de Validation')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)
    axes[0, 1].set_ylim(0, 1)
    
    # 3. Learning Rate
    axes[1, 0].plot(epochs, training_log['learning_rates'], 'orange', linewidth=2)
    axes[1, 0].set_xlabel('Epoch')
    axes[1, 0].set_ylabel('Learning Rate')
    axes[1, 0].set_title('Évolution du Learning Rate')
    axes[1, 0].grid(True, alpha=0.3)
    axes[1, 0].set_yscale('log')
    
    # 4. Comparaison Train vs Validation Loss
    loss_diff = np.array(training_log['val_losses']) - np.array(training_log['train_losses'])
    axes[1, 1].plot(epochs, loss_diff, 'purple', linewidth=2)
    axes[1, 1].axhline(y=0, color='k', linestyle='-', alpha=0.3)
    axes[1, 1].set_xlabel('Epoch')
    axes[1, 1].set_ylabel('Val Loss - Train Loss')
    axes[1, 1].set_title('Écart Train/Validation (Overfitting)')
    axes[1, 1].grid(True, alpha=0.3)
    
    # Coloration selon overfitting
    axes[1, 1].fill_between(epochs, loss_diff, 0, 
                           where=(loss_diff > 0), color='red', alpha=0.3, label='Overfitting')
    axes[1, 1].fill_between(epochs, loss_diff, 0, 
                           where=(loss_diff <= 0), color='green', alpha=0.3, label='Bon apprentissage')
    axes[1, 1].legend()
    
    plt.tight_layout()
    plt.show()
    
    # Analyse des performances
    print(f"\nAnalyse des performances:")
    print(f"{'='*50}")
    
    # Stabilité de l'entraînement
    final_epochs = training_log['val_dice_scores'][-50:]  # Dernières 50 epochs
    stability = np.std(final_epochs) if len(final_epochs) >= 10 else np.std(training_log['val_dice_scores'])
    
    print(f"Stabilité finale (std): {stability:.4f}")
    if stability < 0.01:
        print("✓ Entraînement stable")
    elif stability < 0.03:
        print("⚠ Entraînement moyennement stable")
    else:
        print("✗ Entraînement instable")
    
    # Convergence
    improvement_rate = (training_log['val_dice_scores'][-1] - training_log['val_dice_scores'][0]) / len(epochs)
    print(f"\nTaux d'amélioration: {improvement_rate:.6f} Dice/epoch")
    
    if training_log['val_dice_scores'][-1] > 0.85:
        print("✓ Performance excellente (Dice > 0.85)")
    elif training_log['val_dice_scores'][-1] > 0.75:
        print("✓ Performance bonne (Dice > 0.75)")
    elif training_log['val_dice_scores'][-1] > 0.60:
        print("⚠ Performance modérée (Dice > 0.60)")
    else:
        print("✗ Performance faible (Dice < 0.60)")
    
    # Overfitting
    final_gap = training_log['val_losses'][-1] - training_log['train_losses'][-1]
    print(f"\nÉcart final train/val: {final_gap:.4f}")
    if final_gap < 0.05:
        print("✓ Pas d'overfitting détecté")
    elif final_gap < 0.15:
        print("⚠ Léger overfitting")
    else:
        print("✗ Overfitting significatif")

# Création des visualisations
plot_training_metrics(training_log)

# Tableau récapitulatif des configurations
def create_performance_summary(training_log):
    """
    Crée un résumé des performances
    """
    summary_data = {
        'Configuration': training_log['configuration'],
        'Dataset': f"Dataset{training_log['dataset_id']:03d}",
        'Fold': training_log['fold'],
        'Epochs': training_log['total_epochs'],
        'Meilleur Dice': f"{training_log['best_val_dice']:.4f}",
        'Epoch optimal': training_log['best_epoch'],
        'Loss finale (train)': f"{training_log['final_train_loss']:.4f}",
        'Loss finale (val)': f"{training_log['final_val_loss']:.4f}",
        'Architecture': training_log['architecture']['name']
    }
    
    print(f"\nRésumé de Performance:")
    print(f"{'='*40}")
    for key, value in summary_data.items():
        print(f"{key:<20}: {value}")
    
    return summary_data

performance_summary = create_performance_summary(training_log)

print("\nVisualisation des performances terminée.")

## 6. Inférence et Post-traitement

Utilisons le modèle entraîné pour faire des prédictions sur de nouvelles données.

In [ ]:
# Inférence et post-traitement nnU-Net
print("=== INFÉRENCE ET POST-TRAITEMENT ===")

class nnUNetPredictor:
    """
    Simulateur de prédiction nnU-Net
    """
    
    def __init__(self, model_folder, configuration='3d_fullres'):
        self.model_folder = model_folder
        self.configuration = configuration
        self.postprocessing_enabled = True
        
    def predict_single_case(self, input_image_path, output_folder):
        """
        Prédiction sur un cas individuel
        """
        print(f"Prédiction pour: {input_image_path.name}")
        
        try:
            # Chargement de l'image
            nifti_img = nib.load(input_image_path)
            image_data = nifti_img.get_fdata()
            
            print(f"  Image chargée: {image_data.shape}")
            
            # Simulation du preprocessing
            print(f"  Preprocessing...")
            # Normalisation Z-score
            mean_val = np.mean(image_data[image_data > 0])
            std_val = np.std(image_data[image_data > 0])
            normalized_image = (image_data - mean_val) / (std_val + 1e-8)
            
            # Simulation de la prédiction
            print(f"  Inférence du modèle...")
            
            # Création d'une segmentation simulée réaliste
            prediction = self._simulate_hippocampus_prediction(image_data)
            
            # Post-traitement
            if self.postprocessing_enabled:
                print(f"  Post-traitement...")
                prediction = self._apply_postprocessing(prediction)
            
            # Sauvegarde
            output_path = output_folder / f"{input_image_path.stem.replace('_0000', '')}_predicted.nii.gz"
            output_folder.mkdir(parents=True, exist_ok=True)
            
            predicted_nifti = nib.Nifti1Image(prediction.astype(np.uint8), nifti_img.affine)
            nib.save(predicted_nifti, output_path)
            
            # Calcul de métriques de qualité
            metrics = self._calculate_prediction_metrics(prediction)
            
            print(f"  ✓ Prédiction sauvegardée: {output_path}")
            print(f"  Volume hippocampe gauche: {metrics['left_volume']} voxels")
            print(f"  Volume hippocampe droit: {metrics['right_volume']} voxels")
            print(f"  Score de confiance: {metrics['confidence']:.3f}")
            
            return {
                'input_path': input_image_path,
                'output_path': output_path,
                'prediction_shape': prediction.shape,
                'metrics': metrics
            }
            
        except Exception as e:
            print(f"  ✗ Erreur lors de la prédiction: {e}")
            return None
    
    def _simulate_hippocampus_prediction(self, image_data):
        """
        Simule une prédiction d'hippocampe réaliste
        """
        shape = image_data.shape
        prediction = np.zeros(shape, dtype=np.uint8)
        
        # Coordonnées pour simulation réaliste
        x, y, z = np.meshgrid(
            np.linspace(-1, 1, shape[0]),
            np.linspace(-1, 1, shape[1]),
            np.linspace(-1, 1, shape[2]),
            indexing='ij'
        )
        
        # Hippocampe gauche avec forme réaliste
        left_hippo = (
            (x + 0.3)**2 * 4 + (y - 0.1)**2 * 8 + z**2 * 12 < 0.15
        ) & (image_data > np.percentile(image_data[image_data > 0], 20))
        
        # Hippocampe droit
        right_hippo = (
            (x - 0.3)**2 * 4 + (y - 0.1)**2 * 8 + z**2 * 12 < 0.15
        ) & (image_data > np.percentile(image_data[image_data > 0], 20))
        
        # Ajout de bruit réaliste (erreurs de segmentation)
        noise_mask = np.random.random(shape) < 0.02  # 2% de pixels bruités
        left_hippo = left_hippo & ~noise_mask
        right_hippo = right_hippo & ~noise_mask
        
        prediction[left_hippo] = 1
        prediction[right_hippo] = 2
        
        return prediction
    
    def _apply_postprocessing(self, prediction):
        """
        Applique le post-traitement (nettoyage, connexité)
        """
        from scipy import ndimage
        from skimage import morphology, measure
        
        processed = prediction.copy()
        
        # Post-traitement par label
        for label in [1, 2]:  # Hippocampe gauche et droit
            mask = (prediction == label)
            
            if np.any(mask):
                # Fermeture morphologique
                mask = ndimage.binary_closing(mask, structure=np.ones((3, 3, 3)))
                
                # Suppression des petites composantes
                labeled_mask = measure.label(mask)
                props = measure.regionprops(labeled_mask)
                
                if props:
                    # Garder seulement la plus grande composante
                    largest_component = max(props, key=lambda x: x.area)
                    mask = (labeled_mask == largest_component.label)
                
                processed[prediction == label] = 0
                processed[mask] = label
        
        return processed
    
    def _calculate_prediction_metrics(self, prediction):
        """
        Calcule des métriques de qualité de prédiction
        """
        left_volume = np.sum(prediction == 1)
        right_volume = np.sum(prediction == 2)
        total_volume = left_volume + right_volume
        
        # Score de confiance basé sur la régularité
        if total_volume > 0:
            # Compacité des régions
            props_left = measure.regionprops((prediction == 1).astype(int))
            props_right = measure.regionprops((prediction == 2).astype(int))
            
            compactness = 0
            if props_left:
                compactness += props_left[0].solidity
            if props_right:
                compactness += props_right[0].solidity
            
            confidence = compactness / (2 if props_left and props_right else 1)
        else:
            confidence = 0
        
        return {
            'left_volume': left_volume,
            'right_volume': right_volume,
            'total_volume': total_volume,
            'volume_ratio': left_volume / right_volume if right_volume > 0 else 0,
            'confidence': confidence
        }
    
    def predict_folder(self, input_folder, output_folder):
        """
        Prédiction sur un dossier d'images
        """
        input_files = list(input_folder.glob('*_0000.nii.gz'))
        
        if not input_files:
            print(f"Aucun fichier d'entrée trouvé dans {input_folder}")
            return []
        
        print(f"Prédiction sur {len(input_files)} fichiers...")
        
        results = []
        for img_file in input_files:
            result = self.predict_single_case(img_file, output_folder)
            if result:
                results.append(result)
        
        return results

# Initialisation du prédicteur
predictor = nnUNetPredictor(results_folder, configuration='3d_fullres')

# Test de prédiction sur les données d'entraînement
images_folder = dataset_paths['dataset_folder'] / 'imagesTr'
predictions_folder = results_folder / 'predictions'

print(f"Test de prédiction sur le dataset d'entraînement...")
prediction_results = predictor.predict_folder(images_folder, predictions_folder)

print(f"\nPrédictions terminées: {len(prediction_results)} cas traités")

# Analyse des résultats de prédiction
if prediction_results:
    print(f"\nAnalyse des résultats:")
    print(f"{'='*50}")
    
    volumes_left = [r['metrics']['left_volume'] for r in prediction_results]
    volumes_right = [r['metrics']['right_volume'] for r in prediction_results]
    confidences = [r['metrics']['confidence'] for r in prediction_results]
    
    print(f"Volume hippocampe gauche:")
    print(f"  Moyenne: {np.mean(volumes_left):.0f} ± {np.std(volumes_left):.0f} voxels")
    print(f"  Min/Max: {min(volumes_left)}/{max(volumes_left)} voxels")
    
    print(f"\nVolume hippocampe droit:")
    print(f"  Moyenne: {np.mean(volumes_right):.0f} ± {np.std(volumes_right):.0f} voxels")
    print(f"  Min/Max: {min(volumes_right)}/{max(volumes_right)} voxels")
    
    print(f"\nConfiance moyenne: {np.mean(confidences):.3f}")
    
    # Détection d'anomalies volumétriques
    mean_vol_left = np.mean(volumes_left)
    mean_vol_right = np.mean(volumes_right)
    
    asymmetry_ratio = abs(mean_vol_left - mean_vol_right) / max(mean_vol_left, mean_vol_right)
    print(f"\nAsymétrie hippocampique: {asymmetry_ratio:.3f}")
    
    if asymmetry_ratio < 0.1:
        print("✓ Symétrie normale")
    elif asymmetry_ratio < 0.3:
        print("⚠ Asymétrie légère")
    else:
        print("⚠ Asymétrie significative - investigation recommandée")

print(f"\nInférence nnU-Net terminée.")
print(f"Prédictions disponibles dans: {predictions_folder}")

## 7. Intégration DICOM et Workflows Cliniques

Intégrons nnU-Net dans des workflows hospitaliers réels avec gestion DICOM et connexion PACS.

In [ ]:
# Intégration DICOM et workflows cliniques
print("=== INTÉGRATION DICOM ET WORKFLOWS CLINIQUES ===")

import pydicom
from datetime import datetime

class ClinicalWorkflowIntegrator:
    """
    Intègre nnU-Net dans les workflows hospitaliers
    """
    
    def __init__(self, nnunet_predictor, pacs_config=None):
        self.predictor = nnunet_predictor
        self.pacs_config = pacs_config or {
            'host': '192.168.1.100',
            'port': 104,
            'aec': 'HOSPITAL_PACS',
            'aet': 'NNUNET_AI'
        }
        
        # Configuration clinique
        self.clinical_config = {
            'auto_processing': True,
            'quality_checks': True,
            'radiologist_review': True,
            'report_generation': True
        }
    
    def create_dicom_metadata(self, patient_info, study_info, series_info):
        """
        Crée les métadonnées DICOM pour les segmentations nnU-Net
        """
        metadata = {
            # Informations patient
            'PatientName': patient_info.get('name', 'ANONYMOUS'),
            'PatientID': patient_info.get('id', 'AI_PATIENT_001'),
            'PatientBirthDate': patient_info.get('birth_date', '19800101'),
            'PatientSex': patient_info.get('sex', 'O'),
            
            # Informations étude
            'StudyInstanceUID': study_info.get('study_uid', '1.2.3.4.5.6.7.8.9.1'),
            'StudyDate': study_info.get('study_date', datetime.now().strftime('%Y%m%d')),
            'StudyTime': study_info.get('study_time', datetime.now().strftime('%H%M%S')),
            'StudyDescription': study_info.get('description', 'Brain MRI with nnU-Net Segmentation'),
            'AccessionNumber': study_info.get('accession', 'AI001'),
            
            # Informations série
            'SeriesInstanceUID': series_info.get('series_uid', '1.2.3.4.5.6.7.8.9.2'),
            'SeriesNumber': series_info.get('series_number', 999),
            'SeriesDescription': series_info.get('description', 'nnU-Net Hippocampus Segmentation'),
            'Modality': 'SEG',  # Segmentation Object
            
            # Informations IA
            'ManufacturerModelName': 'nnU-Net v2.0',
            'SoftwareVersions': 'nnU-Net 2.0 + Custom Clinical Integration',
            'DeviceSerialNumber': 'NNUNET_CLINICAL_001'
        }
        
        return metadata
    
    def process_dicom_study(self, dicom_folder, output_folder):
        """
        Traite une étude DICOM complète
        """
        print(f"Traitement de l'étude DICOM: {dicom_folder}")
        
        # Simulation de lecture DICOM
        dicom_files = list(Path(dicom_folder).glob('*.dcm')) if Path(dicom_folder).exists() else []
        
        if not dicom_files:
            print("  Aucun fichier DICOM trouvé, utilisation de données simulées")
            # Simulation d'une étude
            study_results = self._simulate_dicom_study()
        else:
            study_results = self._process_real_dicom_study(dicom_files, output_folder)
        
        return study_results
    
    def _simulate_dicom_study(self):
        """
        Simule le traitement d'une étude DICOM
        """
        print("  Simulation d'une étude DICOM...")
        
        # Informations patient simulées
        patient_info = {
            'name': 'DOE^JOHN',
            'id': 'PAT001',
            'birth_date': '19750315',
            'sex': 'M',
            'age': 48
        }
        
        study_info = {
            'study_uid': f"1.2.826.0.1.3680043.8.498.{int(time.time())}",
            'study_date': datetime.now().strftime('%Y%m%d'),
            'study_time': datetime.now().strftime('%H%M%S'),
            'description': 'IRM cérébrale - Suspicion atrophie hippocampique',
            'accession': f"ACC{datetime.now().strftime('%Y%m%d%H%M')}"
        }
        
        # Simulation du processus clinique
        workflow_steps = [
            'Réception de l\'étude depuis PACS',
            'Conversion DICOM vers NIfTI',
            'Préprocessing nnU-Net', 
            'Inférence de segmentation',
            'Post-traitement et validation qualité',
            'Génération du rapport IA',
            'Intégration dans le PACS'
        ]
        
        print(f"  Patient: {patient_info['name']} (ID: {patient_info['id']})")
        print(f"  Étude: {study_info['description']}")
        print(f"  Numéro d'accession: {study_info['accession']}")
        
        for i, step in enumerate(workflow_steps, 1):
            print(f"  {i}. {step}")
            time.sleep(0.1)  # Simulation du temps de traitement
        
        # Simulation des résultats
        segmentation_results = {
            'hippocampus_left_volume': 2850.5,  # mm³
            'hippocampus_right_volume': 3020.2,  # mm³
            'total_volume': 5870.7,
            'asymmetry_index': 0.058,
            'confidence_score': 0.94,
            'processing_time': 45.2  # secondes
        }
        
        # Évaluation clinique automatique
        clinical_assessment = self._generate_clinical_assessment(segmentation_results, patient_info)
        
        return {
            'patient_info': patient_info,
            'study_info': study_info,
            'segmentation_results': segmentation_results,
            'clinical_assessment': clinical_assessment,
            'workflow_status': 'completed',
            'processing_timestamp': datetime.now().isoformat()
        }
    
    def _generate_clinical_assessment(self, results, patient_info):
        """
        Génère une évaluation clinique automatique
        """
        assessment = {
            'volumetric_analysis': {},
            'normative_comparison': {},
            'clinical_interpretation': {},
            'recommendations': []
        }
        
        # Analyse volumétrique
        left_vol = results['hippocampus_left_volume']
        right_vol = results['hippocampus_right_volume']
        asymmetry = results['asymmetry_index']
        
        assessment['volumetric_analysis'] = {
            'left_hippocampus': f"{left_vol:.1f} mm³",
            'right_hippocampus': f"{right_vol:.1f} mm³",
            'total_volume': f"{results['total_volume']:.1f} mm³",
            'asymmetry_index': f"{asymmetry:.3f}"
        }
        
        # Comparaison normative (valeurs de référence)
        age = patient_info['age']
        sex = patient_info['sex']
        
        # Valeurs normatives approximatives
        if sex == 'M':
            expected_volume = 3200 - (age - 30) * 15  # Déclin avec l'âge
        else:
            expected_volume = 2900 - (age - 30) * 12
        
        volume_percentile = ((left_vol + right_vol) / 2) / expected_volume * 100
        
        assessment['normative_comparison'] = {
            'expected_volume_range': f"{expected_volume-200:.0f}-{expected_volume+200:.0f} mm³",
            'patient_percentile': f"{volume_percentile:.0f}%",
            'age_sex_adjusted': 'Appliqué'
        }
        
        # Interprétation clinique
        if volume_percentile < 10:
            severity = 'Atrophie sévère'
            priority = 'Élevée'
        elif volume_percentile < 25:
            severity = 'Atrophie modérée'
            priority = 'Modérée'
        elif volume_percentile < 75:
            severity = 'Normal'
            priority = 'Routine'
        else:
            severity = 'Au-dessus de la normale'
            priority = 'Routine'
        
        if asymmetry > 0.15:
            asymmetry_status = 'Asymétrie significative'
        elif asymmetry > 0.10:
            asymmetry_status = 'Asymétrie légère'
        else:
            asymmetry_status = 'Symétrie normale'
        
        assessment['clinical_interpretation'] = {
            'volume_assessment': severity,
            'asymmetry_assessment': asymmetry_status,
            'clinical_priority': priority,
            'confidence_level': 'Élevée' if results['confidence_score'] > 0.9 else 'Modérée'
        }
        
        # Recommandations
        if severity in ['Atrophie sévère', 'Atrophie modérée']:
            assessment['recommendations'].extend([
                'Corrélation avec évaluation neuropsychologique',
                'Consultation neurologie/gériatrie', 
                'Suivi volumétrique à 6-12 mois'
            ])
        
        if asymmetry > 0.15:
            assessment['recommendations'].extend([
                'Investigation étiologique de l\'asymétrie',
                'Recherche de lésions focales'
            ])
        
        assessment['recommendations'].append('Validation radiologique requise')
        
        return assessment
    
    def generate_clinical_report(self, study_data):
        """
        Génère un rapport clinique structuré
        """
        patient = study_data['patient_info']
        study = study_data['study_info']
        results = study_data['segmentation_results']
        assessment = study_data['clinical_assessment']
        
        report = f"""
RAPPORT D'ANALYSE VOLUMÉTRIQUE IA - HIPPOCAMPE
{'='*60}

INFORMATIONS PATIENT:
Nom: {patient['name']}
ID: {patient['id']}
Âge: {patient['age']} ans
Sexe: {patient['sex']}

INFORMATIONS ÉTUDE:
Date: {study['study_date']}
Description: {study['description']}
N° Accession: {study['accession']}

RÉSULTATS DE SEGMENTATION IA:
Système: nnU-Net v2.0 + Intégration Clinique
Temps de traitement: {results['processing_time']:.1f} secondes
Score de confiance: {results['confidence_score']:.3f}

ANALYSE VOLUMÉTRIQUE:
• Hippocampe gauche: {assessment['volumetric_analysis']['left_hippocampus']}
• Hippocampe droit: {assessment['volumetric_analysis']['right_hippocampus']}
• Volume total: {assessment['volumetric_analysis']['total_volume']}
• Index d'asymétrie: {assessment['volumetric_analysis']['asymmetry_index']}

COMPARAISON NORMATIVE:
• Plage attendue (âge/sexe): {assessment['normative_comparison']['expected_volume_range']}
• Percentile patient: {assessment['normative_comparison']['patient_percentile']}
• Ajustement: {assessment['normative_comparison']['age_sex_adjusted']}

INTERPRÉTATION CLINIQUE:
• Évaluation volumétrique: {assessment['clinical_interpretation']['volume_assessment']}
• Évaluation asymétrie: {assessment['clinical_interpretation']['asymmetry_assessment']}
• Priorité clinique: {assessment['clinical_interpretation']['clinical_priority']}
• Niveau de confiance: {assessment['clinical_interpretation']['confidence_level']}

RECOMMANDATIONS:
"""
        
        for i, recommendation in enumerate(assessment['recommendations'], 1):
            report += f"{i}. {recommendation}\n"
        
        report += f"""

AVERTISSEMENTS:
• Cette analyse est générée automatiquement par IA
• Validation par radiologue senior requise
• Corrélation avec examen clinique indispensable
• L'IA est un outil d'aide au diagnostic, non substitutif

{'='*60}
Rapport généré le: {datetime.now().strftime('%d/%m/%Y à %H:%M:%S')}
Système: nnU-Net Clinical Integration v1.0
{'='*60}
"""
        
        return report

# Initialisation de l'intégrateur clinique
clinical_integrator = ClinicalWorkflowIntegrator(predictor)

# Simulation d'un workflow clinique complet
print("Test du workflow clinique intégré...")
study_data = clinical_integrator.process_dicom_study("./dicom_input", "./dicom_output")

# Génération du rapport clinique
clinical_report = clinical_integrator.generate_clinical_report(study_data)

print("\n" + "="*60)
print("RAPPORT CLINIQUE GÉNÉRÉ:")
print("="*60)
print(clinical_report)

# Sauvegarde du rapport
report_file = results_folder / f"clinical_report_{study_data['study_info']['accession']}.txt"
with open(report_file, 'w', encoding='utf-8') as f:
    f.write(clinical_report)

print(f"Rapport sauvegardé: {report_file}")
print("\nIntégration clinique nnU-Net complétée.")

## 8. Déploiement et Monitoring en Production

Configurons le déploiement en production avec monitoring des performances et gestion des erreurs.

In [ ]:
# Déploiement et monitoring en production
print("=== DÉPLOIEMENT ET MONITORING EN PRODUCTION ===")

class ProductionDeploymentManager:
    """
    Gestionnaire de déploiement en production pour nnU-Net
    """
    
    def __init__(self, model_path, deployment_config=None):
        self.model_path = model_path
        self.deployment_config = deployment_config or self._default_config()
        self.performance_metrics = {
            'total_processed': 0,
            'successful_predictions': 0,
            'failed_predictions': 0,
            'average_processing_time': 0,
            'uptime_start': datetime.now(),
            'last_error': None
        }
        self.quality_metrics = []
        
    def _default_config(self):
        return {
            'max_concurrent_jobs': 4,
            'max_processing_time': 300,  # 5 minutes
            'auto_restart_on_error': True,
            'backup_frequency': 24,  # heures
            'monitoring_interval': 60,  # secondes
            'alert_thresholds': {
                'error_rate': 0.05,  # 5%
                'processing_time': 180,  # 3 minutes
                'disk_space': 0.9  # 90%
            }
        }
    
    def health_check(self):
        """
        Vérifie l'état du système
        """
        checks = {
            'model_loaded': self.model_path.exists(),
            'gpu_available': torch.cuda.is_available(),
            'disk_space': self._check_disk_space(),
            'memory_usage': self._check_memory_usage(),
            'error_rate': self._calculate_error_rate(),
            'system_uptime': self._get_uptime()
        }
        
        # Évaluation globale
        critical_checks = ['model_loaded', 'disk_space']
        warning_checks = ['gpu_available', 'memory_usage', 'error_rate']
        
        status = 'healthy'
        issues = []
        
        for check in critical_checks:
            if not checks[check]:
                status = 'critical'
                issues.append(f"CRITIQUE: {check} échoué")
        
        for check in warning_checks:
            if not checks[check] and status == 'healthy':
                status = 'warning'
                issues.append(f"ATTENTION: {check} problématique")
        
        return {
            'status': status,
            'timestamp': datetime.now().isoformat(),
            'checks': checks,
            'issues': issues
        }
    
    def _check_disk_space(self):
        """Vérifie l'espace disque disponible"""
        # Simulation
        available_space = 0.75  # 75% disponible
        return available_space > (1 - self.deployment_config['alert_thresholds']['disk_space'])
    
    def _check_memory_usage(self):
        """Vérifie l'usage mémoire"""
        # Simulation
        memory_usage = 0.65  # 65% utilisé
        return memory_usage < 0.85
    
    def _calculate_error_rate(self):
        """Calcule le taux d'erreur"""
        if self.performance_metrics['total_processed'] == 0:
            return True
        
        error_rate = (self.performance_metrics['failed_predictions'] / 
                     self.performance_metrics['total_processed'])
        return error_rate < self.deployment_config['alert_thresholds']['error_rate']
    
    def _get_uptime(self):
        """Calcule le temps de fonctionnement"""
        uptime = datetime.now() - self.performance_metrics['uptime_start']
        return str(uptime)
    
    def process_with_monitoring(self, input_data, case_id):
        """
        Traite une prédiction avec monitoring complet
        """
        start_time = time.time()
        
        try:
            # Mise à jour des métriques
            self.performance_metrics['total_processed'] += 1
            
            # Simulation du traitement
            print(f"Traitement du cas: {case_id}")
            
            # Vérifications préalables
            if not self._validate_input(input_data):
                raise ValueError("Données d'entrée invalides")
            
            # Simulation de la prédiction
            time.sleep(np.random.uniform(1, 3))  # Temps de traitement variable
            
            # Génération des résultats
            results = {
                'case_id': case_id,
                'predictions': {
                    'hippocampus_left': np.random.uniform(2500, 3500),
                    'hippocampus_right': np.random.uniform(2500, 3500)
                },
                'confidence': np.random.uniform(0.85, 0.99),
                'processing_time': time.time() - start_time,
                'timestamp': datetime.now().isoformat()
            }
            
            # Validation qualité
            quality_score = self._assess_prediction_quality(results)
            results['quality_score'] = quality_score
            
            # Mise à jour des métriques de succès
            self.performance_metrics['successful_predictions'] += 1
            self._update_processing_time(results['processing_time'])
            self.quality_metrics.append(quality_score)
            
            print(f"  ✓ Traitement réussi - Temps: {results['processing_time']:.1f}s")
            print(f"  ✓ Qualité: {quality_score:.3f}, Confiance: {results['confidence']:.3f}")
            
            return results
            
        except Exception as e:
            # Gestion des erreurs
            self.performance_metrics['failed_predictions'] += 1
            self.performance_metrics['last_error'] = {
                'error': str(e),
                'case_id': case_id,
                'timestamp': datetime.now().isoformat()
            }
            
            print(f"  ✗ Erreur lors du traitement: {e}")
            
            return {
                'case_id': case_id,
                'error': str(e),
                'timestamp': datetime.now().isoformat(),
                'processing_time': time.time() - start_time
            }
    
    def _validate_input(self, input_data):
        """Valide les données d'entrée"""
        # Simulation de validation
        return np.random.random() > 0.05  # 95% de succès
    
    def _assess_prediction_quality(self, results):
        """Évalue la qualité de la prédiction"""
        # Facteurs de qualité simulés
        confidence_factor = results['confidence']
        volume_consistency = 1 - abs(results['predictions']['hippocampus_left'] - 
                                   results['predictions']['hippocampus_right']) / 3000
        processing_speed = min(1.0, 60 / results['processing_time'])  # Optimal < 60s
        
        quality_score = (confidence_factor * 0.5 + volume_consistency * 0.3 + 
                        processing_speed * 0.2)
        
        return np.clip(quality_score, 0, 1)
    
    def _update_processing_time(self, new_time):
        """Met à jour la moyenne des temps de traitement"""
        current_avg = self.performance_metrics['average_processing_time']
        total_success = self.performance_metrics['successful_predictions']
        
        if total_success == 1:
            self.performance_metrics['average_processing_time'] = new_time
        else:
            # Moyenne mobile
            self.performance_metrics['average_processing_time'] = (
                (current_avg * (total_success - 1) + new_time) / total_success
            )
    
    def generate_monitoring_report(self):
        """
        Génère un rapport de monitoring détaillé
        """
        metrics = self.performance_metrics
        health = self.health_check()
        
        success_rate = (metrics['successful_predictions'] / 
                       metrics['total_processed'] * 100) if metrics['total_processed'] > 0 else 0
        
        avg_quality = np.mean(self.quality_metrics) if self.quality_metrics else 0
        
        report = f"""
RAPPORT DE MONITORING - nnU-Net PRODUCTION
{'='*60}

ÉTAT DU SYSTÈME:
Statut global: {health['status'].upper()}
Dernière vérification: {health['timestamp']}

MÉTRIQUES DE PERFORMANCE:
• Cas traités: {metrics['total_processed']}
• Succès: {metrics['successful_predictions']} ({success_rate:.1f}%)
• Échecs: {metrics['failed_predictions']}
• Temps moyen de traitement: {metrics['average_processing_time']:.1f} secondes
• Score qualité moyen: {avg_quality:.3f}
• Temps de fonctionnement: {health['checks']['system_uptime']}

VÉRIFICATIONS SYSTÈME:
• Modèle chargé: {'✓' if health['checks']['model_loaded'] else '✗'}
• GPU disponible: {'✓' if health['checks']['gpu_available'] else '✗'}
• Espace disque: {'✓' if health['checks']['disk_space'] else '✗'}
• Utilisation mémoire: {'✓' if health['checks']['memory_usage'] else '✗'}
• Taux d'erreur: {'✓' if health['checks']['error_rate'] else '✗'}
"""
        
        if health['issues']:
            report += f"\nPROBLÈMES DÉTECTÉS:\n"
            for issue in health['issues']:
                report += f"• {issue}\n"
        
        if metrics['last_error']:
            error = metrics['last_error']
            report += f"""

DERNIÈRE ERREUR:
• Cas: {error['case_id']}
• Erreur: {error['error']}
• Timestamp: {error['timestamp']}
"""
        
        report += f"""

RECOMMANDATIONS:
{'='*30}
"""
        
        recommendations = self._generate_recommendations(health, metrics)
        for rec in recommendations:
            report += f"• {rec}\n"
        
        report += f"""

{'='*60}
Rapport généré: {datetime.now().strftime('%d/%m/%Y %H:%M:%S')}
{'='*60}
"""
        
        return report
    
    def _generate_recommendations(self, health, metrics):
        """Génère des recommandations basées sur l'état du système"""
        recommendations = []
        
        # Recommandations basées sur l'état
        if health['status'] == 'critical':
            recommendations.append("URGENT: Intervention technique requise")
            recommendations.append("Arrêt temporaire du service recommandé")
        
        # Recommandations basées sur les performances
        if metrics['total_processed'] > 0:
            error_rate = metrics['failed_predictions'] / metrics['total_processed']
            if error_rate > 0.1:
                recommendations.append("Taux d'erreur élevé - Vérifier la qualité des données d'entrée")
        
        if metrics['average_processing_time'] > 120:
            recommendations.append("Temps de traitement élevé - Optimisation nécessaire")
        
        if len(self.quality_metrics) > 10 and np.mean(self.quality_metrics[-10:]) < 0.8:
            recommendations.append("Baisse de qualité récente - Vérifier le modèle")
        
        if not recommendations:
            recommendations.append("Système fonctionnel - Surveillance continue")
            recommendations.append("Sauvegarde programmée des logs")
            recommendations.append("Mise à jour des métriques de performance")
        
        return recommendations

# Initialisation du gestionnaire de déploiement
deployment_manager = ProductionDeploymentManager(results_folder)

print("Simulation d'un déploiement en production...")

# Simulation de traitement de plusieurs cas
test_cases = [
    {'case_id': 'PROD_001', 'patient_id': 'PAT_123'},
    {'case_id': 'PROD_002', 'patient_id': 'PAT_124'},
    {'case_id': 'PROD_003', 'patient_id': 'PAT_125'},
    {'case_id': 'PROD_004', 'patient_id': 'PAT_126'},
    {'case_id': 'PROD_005', 'patient_id': 'PAT_127'}
]

print(f"\nTraitement de {len(test_cases)} cas en production:")
print("-" * 50)

production_results = []
for test_case in test_cases:
    result = deployment_manager.process_with_monitoring(
        input_data=test_case, 
        case_id=test_case['case_id']
    )
    production_results.append(result)
    print()

# Génération du rapport de monitoring
monitoring_report = deployment_manager.generate_monitoring_report()

print("\n" + "="*60)
print("RAPPORT DE MONITORING:")
print("="*60)
print(monitoring_report)

# Sauvegarde du rapport
monitoring_file = results_folder / f"monitoring_report_{datetime.now().strftime('%Y%m%d_%H%M%S')}.txt"
with open(monitoring_file, 'w', encoding='utf-8') as f:
    f.write(monitoring_report)

print(f"Rapport de monitoring sauvegardé: {monitoring_file}")
print("\nDéploiement en production simulé avec succès.")

## Résumé et Guide de Déploiement

### Compétences Acquises

Dans ce notebook complet, vous avez maîtrisé:

1. **Configuration Complète de nnU-Net**
   - Structure de dossiers standardisée
   - Variables d'environnement et configuration
   - Préparation de datasets au format Medical Decathlon

2. **Préprocessing et Planification Automatique**
   - Analyse automatique des propriétés d'images
   - Génération de configurations optimisées (2D, 3D_fullres, 3D_lowres, 3D_cascade)
   - Plans d'entraînement adaptatifs

3. **Entraînement et Monitoring**
   - Simulation réaliste d'entraînement nnU-Net
   - Courbes d'apprentissage et métriques de performance
   - Early stopping et validation croisée

4. **Inférence et Post-traitement**
   - Pipeline de prédiction automatisé
   - Post-traitement morphologique
   - Métriques de qualité et validation

5. **Intégration Clinique DICOM**
   - Workflows hospitaliers complets
   - Gestion des métadonnées DICOM
   - Génération de rapports cliniques automatisés
   - Intégration PACS

6. **Déploiement en Production**
   - Monitoring des performances en temps réel
   - Gestion des erreurs et recovery
   - Health checks automatisés
   - Alertes et recommandations

### Guide de Déploiement Réel

#### Phase 1: Installation et Configuration
```bash
# Installation nnU-Net
pip install nnunet

# Configuration des variables d'environnement
export nnUNet_raw=/data/nnUNet_raw
export nnUNet_preprocessed=/data/nnUNet_preprocessed  
export nnUNet_results=/data/nnUNet_results
```

#### Phase 2: Préparation des Données
```bash
# Préprocessing automatique
nnUNetv2_plan_and_preprocess -d DATASET_ID

# Vérification de la configuration
nnUNetv2_print_plans -d DATASET_ID
```

#### Phase 3: Entraînement
```bash
# Entraînement 5-fold cross-validation
nnUNetv2_train DATASET_ID 3d_fullres 0
nnUNetv2_train DATASET_ID 3d_fullres 1
# ... jusqu'au fold 4

# Recherche de la meilleure configuration
nnUNetv2_find_best_configuration DATASET_ID
```

#### Phase 4: Inférence
```bash
# Prédiction sur nouvelles données
nnUNetv2_predict -i INPUT_FOLDER -o OUTPUT_FOLDER \
    -d DATASET_ID -c 3d_fullres -f 0 1 2 3 4
```

### Recommandations de Production

1. **Infrastructure**:
   - Serveurs GPU dédiés (minimum 16 GB VRAM)
   - Stockage SSD rapide pour les données
   - Redondance et sauvegarde automatique

2. **Sécurité**:
   - Chiffrement des données patients
   - Logs d'audit complets
   - Accès contrôlé et authentification

3. **Validation**:
   - Tests de régression automatisés
   - Validation continue sur données de référence
   - Approval workflow radiologique

4. **Performance**:
   - Monitoring 24/7 des métriques
   - Alertes automatiques en cas de problème
   - Optimisation continue des modèles

### Applications Cliniques Spécialisées

- **Neurologie**: Segmentation hippocampe, tumeurs cérébrales
- **Cardiologie**: Segmentation cardiaque, calcul de fractions d'éjection
- **Radiothérapie**: Délineation d'organes à risque, volumes cibles
- **Chirurgie**: Planification préopératoire, guidage per-opératoire

### Prochaine Étape

Le prochain notebook vous enseignera les **frameworks de sélection et comparaison de modèles IA** pour choisir la meilleure approche selon votre contexte clinique spécifique.